In [7]:
import pandas as pd
import numpy as np
import re
from itertools import product, combinations
from rdkit import Chem
from rdkit.Chem import Descriptors
import networkx as nx
from pyvis.network import Network
from rdkit.Chem.rdChemReactions import PreprocessReaction
from rdkit.Chem import MolFromSmiles, AllChem
from rdkit.Chem import rdChemReactions
import os

In [10]:
reaction_dir = "/home/georgeos/Storage/rxn/"
file_names = os.listdir(reaction_dir)
rxn_files = [reaction_dir + x for x in file_names]
rhea_IDs = [int(x.split(".")[0]) for x in file_names]

In [12]:
rhea2ec = pd.read_csv("rhea2ec.tsv", delimiter="\t",engine="c")

In [13]:
rheadirections = pd.read_csv("rhea-directions.tsv", delimiter="\t",engine="c")

In [14]:
rhea2chebis = pd.read_pickle("rhea_to_ChEBIs.p")

In [15]:
reactions = []
for file, ID in zip(rxn_files,rhea_IDs):
    try:
        reaction = AllChem.ReactionFromRxnFile(file)
        reactants = list(reaction.GetReactants())
        products = list(reaction.GetProducts())
        reactions.append( [reaction, reactants, products, len(reactants), len(products), ID] )
    except:
        pass # sorry

RDKit ERROR: [09:42:21] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Post-condition Violation
RDKit ERROR: Element 'hv' not found
RDKit ERROR: Violation occurred on line 91 in file /home/conda/feedstock_root/build_artifacts/rdkit_1616920171002/work/Code/GraphMol/PeriodicTable.h
RDKit ERROR: Failed Expression: anum > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [09:42:21] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Post-condition Violation
RDKit ERROR: Element 'hv' not found
RDKit ERROR: Violation occurred on line 91 in file /home/conda/feedstock_root/build_artifacts/rdkit_1616920171002/work/Code/GraphMol/PeriodicTable.h
RDKit ERROR: Failed Expression: anum > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [09:42:21] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Post-condition Violation
RDKit ERROR: Element 'hv' not found
RDKit ERROR: Violation occurred on line 91 in file /home/conda/feedstock_root/build_artifacts/rdkit_1616920171002/work/Code/GraphMol/PeriodicTable.h
RDKit ERROR:

In [16]:
reactions = pd.DataFrame(reactions)
pd.DataFrame(reactions).columns = ["rdkit_rxn", "rdkit_reactants","rdkit_products","num_reactants","num_products", "rhea_ID"]

In [17]:
ChEBI_reactants = []
ChEBI_products = []
for x in range(len(reactions)):
    _rhea_ID = reactions["rhea_ID"].iloc[x]
    chebis_in_reaction = rhea2chebis[rhea2chebis["rhea_ids"] == _rhea_ID]["rhea_ChEBIs"].iloc[0]
    _ChEBI_reactants = chebis_in_reaction[reactions["num_reactants"][x]:]
    _ChEBI_products = chebis_in_reaction[:reactions["num_reactants"][x]]
    ChEBI_reactants.append(_ChEBI_reactants)
    ChEBI_products.append(_ChEBI_products)

In [18]:
reactions["ChEBI_reactants"] = ChEBI_reactants
reactions["ChEBI_products"] = ChEBI_products

In [19]:
rhea_master_rxns = []
rhea_bi_rxns = []
for x in range(len(reactions)):
    try:
        _ID = reactions["rhea_ID"].iloc[x]
        _master_rxn = rheadirections[(rheadirections["RHEA_ID_LR"] == _ID) | (rheadirections["RHEA_ID_RL"] == _ID)]["RHEA_ID_MASTER"].iloc[0]
        _bi_rxn = _master_rxn = rheadirections[(rheadirections["RHEA_ID_LR"] == _ID) | (rheadirections["RHEA_ID_RL"] == _ID)]["RHEA_ID_BI"].iloc[0]
        rhea_master_rxns.append(_master_rxn)
        rhea_bi_rxns.append(_bi_rxn)
    except:
        rhea_master_rxns.append(np.nan)
        rhea_bi_rxns.append(np.nan)

In [20]:
reactions["RHEA_ID_MASTER"] = rhea_master_rxns
reactions["RHEA_ID_BI"] = rhea_bi_rxns
reactions = reactions[~reactions["RHEA_ID_MASTER"].isna()]
reactions = reactions[~reactions["RHEA_ID_BI"].isna()]


In [21]:
reactions = reactions.drop_duplicates(subset="RHEA_ID_MASTER",keep='first')

In [22]:
reactions = reactions.loc[(reactions["ChEBI_reactants"].astype(str) + reactions["ChEBI_products"].astype(str)).drop_duplicates(keep="first").index]
reactions = reactions[~(reactions["ChEBI_reactants"].astype(str) == reactions["ChEBI_products"].astype(str))]

In [23]:
reactions.reset_index(inplace=True, drop=True)

In [24]:
adjacency_list = []
rhea_id = []
for x in range(len(reactions)):
    mini_adjacency = list(product(reactions["ChEBI_reactants"][x],reactions["ChEBI_products"][x]))
    adjacency_list.extend(mini_adjacency)
    for _ in range(len(mini_adjacency)):
        rhea_id.append(reactions["rhea_ID"][x])

In [29]:
adjacency_list = pd.DataFrame(adjacency_list)
adjacency_list.columns = ["ChEBI_reactant", "ChEBI_product"]
adjacency_list["rhea_ID"] = rhea_id
adjacency_list.reset_index(inplace=True,drop=True)

ValueError: Length mismatch: Expected axis has 3 elements, new values have 2 elements

In [35]:
suppl = Chem.SDMolSupplier("ChEBI_complete_3star.sdf")
ChEBI_molecules = []
for mol in suppl:
    ChEBI_molecules.append(mol)
ChEBI_mol_info = []
for molecule in ChEBI_molecules:
    if molecule is None:
        pass
    else:
        ChEBI_mol_info.append((molecule.GetPropsAsDict()))
ChEBI_mol_info = pd.DataFrame(ChEBI_mol_info)

RDKit WARNING: [10:14:34] WARNING: not removing hydrogen atom without neighbors
RDKit ERROR: rtifacts/rdkit_1616920171002/work/Code/GraphMol/PeriodicTable.h
RDKit ERROR: Failed Expression: anum > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [09:42:32] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Post-condition Violation
RDKit ERROR: Element 'hv' not found
RDKit ERROR: Violation occurred on line 91 in file /home/conda/feedstock_root/build_artifacts/rdkit_1616920171002/work/Code/GraphMol/PeriodicTable.h
RDKit ERROR: Failed Expression: anum > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [09:42:33] 
RDKit ERROR: 
RDKit ERROR: ****
RDKit ERROR: Post-condition Violation
RDKit ERROR: Element 'hv' not found
RDKit ERROR: Violation occurred on line 91 in file /home/conda/feedstock_root/build_artifacts/rdkit_1616920171002/work/Code/GraphMol/PeriodicTable.h
RDKit ERROR: Failed Expression: anum > -1
RDKit ERROR: ****
RDKit ERROR: 
RDKit ERROR: [09:42:33] 
RDKit ERROR: 
RDKit ERROR: ****
RDKi

In [57]:
ChEBI_mol_info["ChEBI ID"] = ChEBI_mol_info["ChEBI ID"].str.split(":").str[-1]
ChEBI_mol_info["Secondary ChEBI ID"] = ChEBI_mol_info["Secondary ChEBI ID"].str.split(":").str[-1]

In [69]:
ChEBI_mol_info = ChEBI_mol_info[~ChEBI_mol_info["ChEBI ID"].isna()]
ChEBI_mol_info = ChEBI_mol_info[~ChEBI_mol_info["Secondary ChEBI ID"].isna()]

In [73]:
ChEBI_mol_info["ChEBI ID"] = ChEBI_mol_info["ChEBI ID"].astype(int)
ChEBI_mol_info["Secondary ChEBI ID"] = ChEBI_mol_info["Secondary ChEBI ID"].astype(int)

In [92]:
CHEBI_names = pd.read_csv("ChEBI_names.tsv",delimiter="\t")

In [119]:
reactant_names = []
for reactant in adjacency_list["ChEBI_reactant"]:
    name = CHEBI_names[CHEBI_names["COMPOUND_ID"] == reactant]["NAME"]
    if len(name) > 0:
        reactant_names.append(name.iloc[0])
    else:
        reactant_names.append(np.nan)
        
product_names = []
for reactant in adjacency_list["ChEBI_product"]:
    name = CHEBI_names[CHEBI_names["COMPOUND_ID"] == reactant]["NAME"]
    if len(name) > 0:
        product_names.append(name.iloc[0])
    else:
        product_names.append(np.nan)

In [120]:
adjacency_list["reactant_name"] = reactant_names
adjacency_list["product_names"] = product_names

In [122]:
molecular_blacklist = [15422, 15751, 16960, 48584, 456216, 61506, 16761, 456215, 15378,15377,30616, 15379,57783,58349,57945]

In [123]:
adjacency_list = adjacency_list[~adjacency_list["ChEBI_reactant"].isin(molecular_blacklist)]
adjacency_list = adjacency_list[~adjacency_list["ChEBI_product"].isin(molecular_blacklist)]

In [124]:
adjacency_list = adjacency_list.drop_duplicates(subset=['ChEBI_reactant', 'ChEBI_product'],keep="first") # fixing stoichiometry issues

In [126]:
uniprot_rhea_EC = pd.read_csv("uniprot_rhea_EC.tab",delimiter="\t")

In [134]:
uniprot_rhea_EC["Rhea ID"] = uniprot_rhea_EC["Rhea ID"].str.split(":").str[-1]

In [137]:
uniprot_rhea_EC = uniprot_rhea_EC[~uniprot_rhea_EC["Rhea ID"].isna()]

In [139]:
uniprot_rhea_EC["Rhea ID"] = uniprot_rhea_EC["Rhea ID"].astype(int)

<ipython-input-139-dcf1213a63dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniprot_rhea_EC["Rhea ID"] = uniprot_rhea_EC["Rhea ID"].astype(int)


In [157]:
EC_numbers = uniprot_rhea_EC['EC number'].str.split('.', expand=True)[[0,1,2,3]]
EC_numbers.columns = ["EC1", "EC2","EC3","EC4"]

In [165]:
uniprot_rhea_EC = pd.concat([uniprot_rhea_EC,EC_numbers],axis=1)

In [188]:
len(uniprot_rhea_EC[uniprot_rhea_EC["Rhea ID"] == 49741])

0

In [217]:
adjacency_list["rhea_ID"]

0        49741
1        49741
2        53150
3        53150
4        53150
         ...  
45459    41802
45460    33521
45461    33521
45462    33521
45463    61262
Name: rhea_ID, Length: 45464, dtype: int64

In [224]:
uniprot_rhea_EC[uniprot_rhea_EC["Rhea ID"] == 49741]

,Entry,EC number,Rhea ID,EC1,EC2,EC3,EC4


In [225]:
EC_numbers = []
for rhea in adjacency_list["rhea_ID"]:
    if (len(uniprot_rhea_EC[uniprot_rhea_EC["Rhea ID"] == rhea])) == 0:
        EC_numbers.append(np.nan)
    else:
        EC_numbers.append(uniprot_rhea_EC[uniprot_rhea_EC["Rhea ID"] == rhea].iloc[0][["EC number"]].iloc[0])

In [210]:
EC_numbers = pd.DataFrame(EC_numbers)
EC_numbers.columns ["rhea_ID", "EC number"]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [208]:
pd.DataFrame(EC_numbers).notna()

,0
0,False
1,False
2,False
3,False
4,False
...,...
45459,False
45460,False
45461,False
45462,False


In [125]:
adjacency_list.to_csv("adjacency_list.csv")

In [98]:
import networkx as nx

In [108]:
G = nx.from_pandas_edgelist(adjacency_list, source="ChEBI_reactant", target="ChEBI_product",edge_attr="rhea_ID")
remove = [node for node,degree in dict(G.degree()).items() if degree > 300]
G.remove_nodes_from(remove)

In [112]:
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt
G.remove_nodes_from(remove)

In [113]:
nx.write_graphml(G,"Storage/g.graphml")